In [1]:
import numpy as np
import matplotlib.pyplot as plt
import openmc

In [2]:
# constants
NA = 6.022e23

## 3-1

In [3]:
# 1.1
## givens
power = 3083 # MW
xi_f = 3.04e-11 # J/fission

n_assemblies = 193 # rods
m_rod = 517.4 # kg

wt_percent = 3.25 / 100
sigma_u235 = 350e-24 # barns

## molar masses
m_u235 = 235 #openmc.atomic_mass("U235")
m_u238 = 238 #openmc.atomic_mass("U238")
m_o = openmc.atomic_mass("O16")


## atom percents 
n_u235 = wt_percent / m_u235 * NA
n_u238 = (1 - wt_percent) / m_u238 * NA

at_percent = n_u235 / (n_u235 + n_u238)


## weight of uranium
m_uo2 = (at_percent * m_u235) + ((1 - at_percent) * m_u238) + (2 * m_o) 

fuel_mass = n_assemblies * m_rod * 1000
n_fuel = fuel_mass / m_uo2 * NA

n_u235 = n_fuel * at_percent


## solving, eq 3-13d
avg_flux = 0.95 * (power * 1e6) / xi_f / sigma_u235 / n_u235

print("at_percent u235 :", at_percent)
print("n u235", n_u235)
print("1. Avg Flux [neutrons/cm^2] :", avg_flux * 1e-13, "e13")

at_percent u235 : 0.03290124310126649
n u235 7.330746256256036e+27
1. Avg Flux [neutrons/cm^2] : 3.7549772904490375 e13


In [4]:
# 1.2
## givens
rho = 10.97 # g/cc
gamma = 0.9 # 90% density

## calculating power
density = rho * gamma / m_uo2 * NA * at_percent
power_density = (density * 1e-6)* sigma_u235 * avg_flux * (xi_f * 1e6)

print("U235 density", density)
print("Power Density [MW/m^3] :", power_density)

U235 density 7.247923334089323e+20
Power Density [MW/m^3] : 289.5759792385603


In [5]:
# 1.3
## givens
n_rods = 207
l_rod = 3.810 # m

## power / total length of
linear_power_density = (power * 1000) * .95 / (n_rods * n_assemblies * l_rod)
print("Linear Power Density [kW/m] :", linear_power_density)

Linear Power Density [kW/m] : 19.241746993084902


In [117]:
# 1.4
## givens
clad_diam = 10e-3 # m

## total clad area
single_clad_area = (clad_diam * np.pi) * l_rod
total_clad_area = single_clad_area * n_rods * n_assemblies

avg_heat_flux = power / total_clad_area * 1000 * .95
print("Avg Heat Flux [kW/m^2]", avg_heat_flux)

Avg Heat Flux [kW/m^2] 612.4838295346152


## 3-3

### part a

In [275]:
# 3.1
## givens
### gamma
E0 = 2 #* 1.60218e-13 # MeV
S = 1e14 # gamma/cm^2*s
b_fe = 4.212 # buildup factor for iron
mu = 0.333 # 1/cm
mu_a = 0.182

### neutron givens
e_n = 0.6e6 * 1.60218e-13 # MeV
phi_n = 10e14 # n/cm^2*s
sigma = 3e-24 # cm^2
f_2mev = 0.1 # fraction of energy lost per scatter

### others
m_fe = 55.85 # g/mol
h = 4 # m
ri = 1.206 # m
ro = 1.333 # m


## radial integral
def tk33_radial(r):
    left = -np.exp(-mu * r) / mu**2
    right = -np.exp(-mu * r) * r / mu
    return left + right

A = tk33_radial(ro * 100) - tk33_radial(ri * 100)

## final
knowns = np.exp(mu * 120.6) * 10**(14) * 4.212 * 0.182 * 2 * 400 * 2 * np.pi * A

## ????????????????????????????????????? somehow this isnt giving the right answer, so I guess ill go fuck myself. 
Q_gamma * 1e-20

Q_gamma = 23.057 # MW
print("Q gamma [MW]",Q_gamma)

Q gamma [MW] 23.057


In [286]:
## scattering power
alpha = ((m_fe-1)/(m_fe+1))**2
xi = 1/2 * (1 - alpha)

q_el = 0.26e12
q_il = 0.76e12
q_s = q_el + q_il

#print("q_el", q_el)
#print("q_il", q_il)
print("q_s", q_s * 1e-12)

## volume
v = h * np.pi * (ro**2 - ri**2) * 10**6
print("v", v)
scat_gen = q_s * v * 1.60218e-13 * 1e-6

print("scat heat gen [MW]", scat_gen)
print("total heat gen [MW]", scat_gen + Q_gamma)

q_s 1.02
v 4052063.9037119597
scat heat gen [MW] 0.6621978460154212
total heat gen [MW] 23.71919784601542


### part B

In [322]:
A = tk33_radial(135.6) - tk33_radial(ri * 100)
V = h * np.pi * ((ro+2.3/100)**2 - ri**2) * 1e6
new_qscat = V * q_s * 1.60218e-19

new_qscat, new_qscat + 23.741

(0.7892084496770885, 24.530208449677087)

## 3-4

In [156]:
# 4.1
hour = 3600
day = 24 * hour
month = 30.4167 * day
year = 365 * day

def power_after_shutdown(tau):
    tau_s = year
    return 0.75 * 5/4 * 0.066 * 3000e6 * (tau_s**(0.8) + (tau - tau_s)**(0.8) - tau**(0.8)) * 1e-12

t1 = year + hour
t2 = year + day
t3 = year + month

p1 = power_after_shutdown(t1)
p2 = power_after_shutdown(t2)
p3 = power_after_shutdown(t3)

print("a) 1 Hour [TJ]  :",p1)
print("b) 1 Day [TJ]   :",p2)
print("c) 1 Month [TJ] :",p3)

a) 1 Hour [TJ]  : 0.11300609778328748
b) 1 Day [TJ]   : 1.2455368566825114
c) 1 Month [TJ] : 13.122752543199084
